In [10]:
from configparser import ConfigParser
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import pickle

parser = ConfigParser()
parser.read('./spotify_credentials.cfg')

SPOTIPY_CLIENT_ID = parser.get('spotify', 'SPOTIPY_CLIENT_ID')
SPOTIPY_CLIENT_SECRET = parser.get('spotify', 'SPOTIPY_CLIENT_SECRET')

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))

user_id = 'czdoifmfngjhvoetavlok9dg5'
playlists = sp.user_playlists(user_id)
playlists = playlists['items']





playlist_id = '5odVaQ10ISGhvuakjVGmxp'





results = sp.playlist_tracks(playlist_id)
tracks = results['items']
track_ids = []
for track in tracks:
    track_ids.append(track['track']['id'])
track_ids

columns_to_remove = ['analysis_url', 'type', 'uri', 'track_href']
headings = ['danceability',
'energy',
'key',
'loudness',
'mode',
'speechiness',
'acousticness',
'instrumentalness',
'liveness',
'valence',
'tempo',
'id',
'duration_ms',
'time_signature',
'track_title',
'album_title',
'album_artist',
'track_number',
'total_tracks']

playlist_data = pd.DataFrame(columns=headings)

playlist_length = len(track_ids)

i = 0
for track in track_ids:
    results = sp.audio_features(track)
    if results[0]: 
        features = results[0]
    
    features_matrix = pd.DataFrame.from_records(features, index=[0])
    
    # Remove unneeded columns
    features_matrix.drop(columns = columns_to_remove, axis = 1, inplace = True)
    # print(features_matrix)

    # Add track title
    track_title = tracks[i]['track']['name']
    features_matrix['track_title'] = track_title

    # Add album title
    album_title = tracks[i]['track']['album']['name']
    features_matrix['album_title'] = album_title

    # Add artist name
    name = tracks[i]['track']['album']['artists'][0]['name']
    features_matrix['album_artist'] = name

    # Add track number and total tracks
    features_matrix['track_number'] = tracks[i]['track']['track_number']
    features_matrix['total_tracks'] = playlist_length

    # Add album as a row to the data df
    # print(features_matrix,'\n')
    # print(data)
    
    playlist_data = pd.concat([playlist_data, features_matrix])
    i += 1

playlist_data = playlist_data.reset_index(drop=True)
# playlist_data.to_csv('./data/playlist_data.csv')
playlist_data_full = pd.DataFrame(playlist_data)



# Remove unnecessary testing columns
columns_to_remove = ['id',
                    'track_title',
                    'album_title',
                    'album_artist',
                    'track_number',
                    'total_tracks',
                    'key',
                    'mode',
                    'duration_ms',
                    'time_signature']
playlist_data.drop(columns = columns_to_remove, axis = 1, inplace = True)


# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))


## Load trained model
model = pickle.load(open('./regression/bayesian.sav', 'rb'))
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
# playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full


# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator BayesianRidge from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


{'snapshot_id': 'MTIsNWZjZTE5NzcxZmJiNGYxODUwNDRhMWFiNDQwYmQ3YmRlODg4ZjY5Nw=='}

In [11]:
## Load trained model
model = pickle.load(open('./regression/en.sav', 'rb'))
playlist_data_full['order'] = model.predict(playlist_data.values)
playlist_data_full = playlist_data_full.sort_values(by=['order'])
playlist_data_full['order'] = np.arange(1, playlist_data_full.shape[0]+1) # convert order to integer playlist track number
playlist_data_full

C:\Users\crisp\anaconda3\envs\AI_general\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator ElasticNet from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
3,0.643,0.776,9,-4.644,1,0.0585,0.152,0.00239,0.0809,0.339,90.979,1iUqqJrrk98kuvtLM0yfA7,181920,4,Running,GALLERY,A R I Z O N A,9,12,1
4,0.652,0.784,0,-4.102,1,0.0398,0.248,0.000042,0.0851,0.445,100.995,0AS090YJNo3vh75GrFxwbX,198493,4,Electric Touch,GALLERY,A R I Z O N A,7,12,2
5,0.631,0.793,2,-4.732,1,0.109,0.0106,0,0.466,0.334,93.89,6se160dvONwrpI4G51bnVE,199960,4,Annie,GALLERY,A R I Z O N A,1,12,3
1,0.651,0.747,0,-5.456,1,0.0382,0.0848,0.000555,0.122,0.476,93.048,4FjPJhlEIWB9QVKFTc78Nn,218760,4,Cross My Mind,GALLERY,A R I Z O N A,6,12,4
2,0.68,0.798,3,-4.978,1,0.116,0.0929,0,0.0746,0.594,116.989,5j4mKTvOu0cr4cooq57sm3,186800,4,Where I Wanna Be,GALLERY,A R I Z O N A,4,12,5
0,0.705,0.782,7,-5.836,0,0.0409,0.00746,0.000684,0.3,0.682,107.019,2driOHZZnMIlaUIt9e3SRD,201947,4,Feed the Beast,GALLERY,A R I Z O N A,3,12,6
9,0.628,0.817,3,-5.298,1,0.0428,0.0384,0.000064,0.0873,0.19,133.025,2PaNQL6MhyTLquEuvdNKbo,226640,4,Brave Enough,GALLERY,A R I Z O N A,8,12,7
7,0.658,0.658,2,-4.866,1,0.0443,0.0628,0,0.0497,0.582,152.078,76viqO3tGyWHYbJgmV2gfN,207400,4,Ain't Gonna Leave,GALLERY,A R I Z O N A,11,12,8
8,0.617,0.642,8,-8.624,1,0.0334,0.13,0.157,0.0929,0.221,105.885,6A8dnC0xkiuWN4BshmTB2I,194013,4,Oceans Away,GALLERY,A R I Z O N A,12,12,9
10,0.626,0.508,6,-7.025,1,0.0285,0.157,0.000008,0.107,0.0778,139.021,3wqUIEuO44HNZxRGlaOMWq,184093,4,Are You,GALLERY,A R I Z O N A,10,12,10


In [8]:
from sklearn.preprocessing import MinMaxScaler
segmentColumnsToNormalise = ['track_number','order']
playlist_data_full[segmentColumnsToNormalise] = MinMaxScaler().fit_transform(playlist_data_full[segmentColumnsToNormalise])
playlist_data_full

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,track_title,album_title,album_artist,track_number,total_tracks,order
1,0.651,0.747,0,-5.456,1,0.0382,0.0848,0.000555,0.122,0.476,93.048,4FjPJhlEIWB9QVKFTc78Nn,218760,4,Cross My Mind,GALLERY,A R I Z O N A,0.454545,12,0.000000
2,0.68,0.798,3,-4.978,1,0.116,0.0929,0,0.0746,0.594,116.989,5j4mKTvOu0cr4cooq57sm3,186800,4,Where I Wanna Be,GALLERY,A R I Z O N A,0.272727,12,0.090909
0,0.705,0.782,7,-5.836,0,0.0409,0.00746,0.000684,0.3,0.682,107.019,2driOHZZnMIlaUIt9e3SRD,201947,4,Feed the Beast,GALLERY,A R I Z O N A,0.181818,12,0.181818
10,0.626,0.508,6,-7.025,1,0.0285,0.157,0.000008,0.107,0.0778,139.021,3wqUIEuO44HNZxRGlaOMWq,184093,4,Are You,GALLERY,A R I Z O N A,0.818182,12,0.272727
5,0.631,0.793,2,-4.732,1,0.109,0.0106,0,0.466,0.334,93.89,6se160dvONwrpI4G51bnVE,199960,4,Annie,GALLERY,A R I Z O N A,0.000000,12,0.363636
4,0.652,0.784,0,-4.102,1,0.0398,0.248,0.000042,0.0851,0.445,100.995,0AS090YJNo3vh75GrFxwbX,198493,4,Electric Touch,GALLERY,A R I Z O N A,0.545455,12,0.454545
7,0.658,0.658,2,-4.866,1,0.0443,0.0628,0,0.0497,0.582,152.078,76viqO3tGyWHYbJgmV2gfN,207400,4,Ain't Gonna Leave,GALLERY,A R I Z O N A,0.909091,12,0.545455
8,0.617,0.642,8,-8.624,1,0.0334,0.13,0.157,0.0929,0.221,105.885,6A8dnC0xkiuWN4BshmTB2I,194013,4,Oceans Away,GALLERY,A R I Z O N A,1.000000,12,0.636364
3,0.643,0.776,9,-4.644,1,0.0585,0.152,0.00239,0.0809,0.339,90.979,1iUqqJrrk98kuvtLM0yfA7,181920,4,Running,GALLERY,A R I Z O N A,0.727273,12,0.727273
6,0.824,0.502,6,-9.38,1,0.0349,0.196,0.0559,0.105,0.563,118.006,3a2MbZSw3NGRWHqsCJ4iQj,217293,4,I Was Wrong,GALLERY,A R I Z O N A,0.090909,12,0.818182


In [9]:
from sklearn.metrics import mean_squared_error
y_true = playlist_data_full['track_number']
y_pred = playlist_data_full['order']
mean_squared_error(y_true, y_pred)

0.16253443526170797

Rearrange

In [25]:
# get permissions to rearrange
from spotipy.oauth2 import SpotifyOAuth
scope = "playlist-modify-public"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope,redirect_uri='http://localhost:5678/',client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET))


# rearrange
sorted_ids = list(playlist_data_full['id'])

UPDATED_PLAYLIST = sp.playlist_replace_items(playlist_id,sorted_ids)
UPDATED_PLAYLIST

{'snapshot_id': 'NjEsOTNlNmM4N2I3OTE3MThmYTM1YWMzOTE5NjJmYzhhN2FkZDI0ZWJhNQ=='}